In [1]:
import requests
from bs4 import BeautifulSoup

#SITE = 'https://dom.ria.com/uk/prodazha-kvartir/kiev/?commercial=all&page='
SITE = 'https://flatfy.ua/uk/%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B6-%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80-%D0%BA%D0%B8%D1%97%D0%B2?page=1'

In [102]:
import pandas as pd

In [2]:
HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }


In [3]:
r = requests.get(SITE, headers = HEADERS)


In [4]:
soup = BeautifulSoup(r.text, 'lxml')

In [5]:
substr = 'поверх'
for el in soup.find_all('span',class_ = 'realty-preview__info'):
    if substr in el.text:
        print(el.text)

поверх 1 з 12
поверх 2 з 9
поверх 4 з 32
поверх 2 з 5
поверх 16 з 24
поверх 1 з 6
поверх 3 з 24
поверх 5 з 10
поверх 9 з 18
поверх 3 з 24
поверх 16 з 24
поверх 11 з 18
поверх 18 з 24
поверх 16 з 16
поверх 9 з 25
поверх 11 з 30
поверх 1 з 21
поверх 9 з 24
поверх 10 з 12
поверх 13 з 16
поверх 7 з 25
поверх 4 з 5
поверх 13 з 24


In [34]:
import pickle

In [6]:
with open('districts.pickle', 'rb') as f:
    second_dict = pickle.load(f)       #load - загружає структуру із файлу
    
second_dict

NameError: name 'pickle' is not defined

In [7]:
%pip install geopandas
%pip install geopy

Note: you may need to restart the kernel to use updated packages.Collecting geopandas
  Using cached geopandas-0.9.0-py2.py3-none-any.whl (994 kB)
  Using cached Shapely-1.7.1-cp38-cp38-win_amd64.whl (1.0 MB)
  Using cached Fiona-1.8.19.tar.gz (1.3 MB)



    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\olga\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\olga\\AppData\\Local\\Temp\\pip-install-mlubn06q\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\olga\\AppData\\Local\\Temp\\pip-install-mlubn06q\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\olga\AppData\Local\Temp\pip-pip-egg-info-jeh0aj3s'
         cwd: C:\Users\olga\AppData\Local\Temp\pip-install-mlubn06q\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install geocoder

import geocoder

In [19]:
g = geocoder.arcgis('вул. Лейпцизька, 13,Київ')

In [20]:
g.latlng

[50.431210003062716, 30.549470032821176]

In [112]:
import re
def conv_coordinates(lat):
    deg, minutes, seconds, direction =  re.split('[°\′″]', lat)
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)

In [32]:
SITE = 'https://en.wikipedia.org/wiki/Zhytomyrska_(Kyiv_Metro)'

In [33]:
r = requests.get(SITE, headers = HEADERS)

In [34]:
soup = BeautifulSoup(r.text, 'lxml')

In [39]:
lat = soup.find('span',class_ = 'latitude')
lat.text

'50°27′21″N'

In [40]:
conv_coordinates(lat.text)

50.45583333333334


In [38]:
soup.find('span',class_ = 'longitude')

<span class="longitude">30°21′51″E</span>

In [41]:
SITE = 'https://en.wikipedia.org/wiki/Sviatoshynsko%E2%80%93Brovarska_line'

In [42]:
r = requests.get(SITE, headers = HEADERS)
soup = BeautifulSoup(r.text, 'lxml')

In [59]:
metro_station = []
for li in soup.find_all('li'):
    try:
        metro = li.a.get('href')
        if '(Kyiv_Metro)'in metro:
            metro_station.append(metro)
    except:
        None
print(len(metro_station))

72


In [60]:
metro_link = set(metro_station)

In [85]:
metro_link = list(metro_link)

In [115]:
SITE = 'https://en.wikipedia.org'
metro_lat = []
for i in metro_link:
    r = requests.get(SITE+i, headers=HEADERS)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'lxml')
        el1 = soup.find('span', class_='latitude').text
        el2 = soup.find('span', class_='longitude').text
        station = soup.find('h1', class_='firstHeading').text
        el1 = conv_coordinates(el1)
        el2 = conv_coordinates(el2)
        
        
        coord = (station,el1,el2)
        
        metro_lat.append(coord)
metro_lat

[('Sviatoshyn (Kyiv Metro)', 50.4575, 30.391944444444444),
 ('Vystavkovyi Tsentr (Kyiv Metro)', 50.3825, 30.4775),
 ('Khreshchatyk (Kyiv Metro)', 50.447222222222216, 30.522777777777776),
 ('Dorohozhychi (Kyiv Metro)', 50.47361111111111, 30.449166666666667),
 ('Syrets (Kyiv Metro)', 50.47638888888889, 30.430833333333336),
 ('Kontraktova Ploshcha (Kyiv Metro)', 50.46527777777778, 30.516666666666666),
 ('Zoloti Vorota (Kyiv Metro)', 50.44833333333333, 30.513333333333332),
 ('Vokzalna (Kyiv Metro)', 50.44166666666666, 30.488055555555558),
 ('Maidan Nezalezhnosti (Kyiv Metro)', 50.45, 30.524444444444445),
 ('Palats Sportu (Kyiv Metro)', 50.43805555555555, 30.520833333333332),
 ('Chervonyi Khutir (Kyiv Metro)', 50.40888888888889, 30.694444444444446),
 ('Beresteiska (Kyiv Metro)', 50.45861111111111, 30.419722222222223),
 ('Vyrlytsia (Kyiv Metro)', 50.40333333333333, 30.66611111111111),
 ('Pochaina (Kyiv Metro)', 50.48694444444445, 30.497777777777777),
 ('Teremky (Kyiv Metro)', 50.367044444444

In [117]:
metro = pd.DataFrame(metro_lat)

In [118]:
import pickle
with open('metro.pickle', 'wb') as f:     #pickle працює в бінарному режимі 
    pickle.dump(metro, f)

In [97]:
len(metro_lat),len(metro_link)

(54, 54)

In [65]:
metro_link

{'/wiki/Akademmistechko_(Kyiv_Metro)',
 '/wiki/Arsenalna_(Kyiv_Metro)',
 '/wiki/Beresteiska_(Kyiv_Metro)',
 '/wiki/Boryspilska_(Kyiv_Metro)',
 '/wiki/Chernihivska_(Kyiv_Metro)',
 '/wiki/Chervonyi_Khutir_(Kyiv_Metro)',
 '/wiki/Darnytsia_(Kyiv_Metro)',
 '/wiki/Demiivska_(Kyiv_Metro)',
 '/wiki/Dnipro_(Kyiv_Metro)',
 '/wiki/Dorohozhychi_(Kyiv_Metro)',
 '/wiki/Druzhby_Narodiv_(Kyiv_Metro)',
 '/wiki/Heroiv_Dnipra_(Kyiv_Metro)',
 '/wiki/Holosiivska_(Kyiv_Metro)',
 '/wiki/Hydropark_(Kyiv_Metro)',
 '/wiki/Ipodrom_(Kyiv_Metro)',
 '/wiki/Kharkivska_(Kyiv_Metro)',
 '/wiki/Khreshchatyk_(Kyiv_Metro)',
 '/wiki/Klovska_(Kyiv_Metro)',
 '/wiki/Kontraktova_Ploshcha_(Kyiv_Metro)',
 '/wiki/Lisova_(Kyiv_Metro)',
 '/wiki/Livoberezhna_(Kyiv_Metro)',
 '/wiki/Lukianivska_(Kyiv_Metro)',
 '/wiki/Lvivska_Brama_(Kyiv_Metro)',
 '/wiki/Lybidska_(Kyiv_Metro)',
 '/wiki/Maidan_Nezalezhnosti_(Kyiv_Metro)',
 '/wiki/Minska_(Kyiv_Metro)',
 '/wiki/Nyvky_(Kyiv_Metro)',
 '/wiki/Obolon_(Kyiv_Metro)',
 '/wiki/Olimpiiska_(Kyiv_Me